In [3]:
import numpy as np
import math
import pandas as pd

In [17]:
def initParams(a: float, b: float, n: int, diferSecond):
    """
    Иницилизирует начальные данные
    :param a: левая граница
    :param b: правая граница
    :param n: количество разбиений
    :param diferSecond: лямбда второй производной
    :return: Возврашает проинецилизорованые данные
    """
    left_matrix = np.zeros((n, n))
    center_matrix = np.zeros(n)
    right_matrix = np.zeros(n)
    lambda_array = np.zeros(n)
    u_array = np.zeros(n)
    h = (b - a) / (n-1)

    x_matrix = np.zeros(n)
    for i in range(n):
        x_matrix[i] = (a + i * h)
    left_matrix[0, 0] = 1
    left_matrix[0, 1] = 0
    left_matrix[- 1, -2] = 0
    left_matrix[- 1, -1] = 1
    right_matrix[0] = diferSecond(a)
    right_matrix[-1] = diferSecond(b)

    return left_matrix, center_matrix, right_matrix, u_array, h, x_matrix, lambda_array


In [18]:
def splainThird(x, m, h, n, value, fun):
    """

    :param x: матрица левая
    :param m: матрица решений
    :param h: шаг
    :param n: количество разбиений
    :param value: искомое значение
    :param fun: исходная функция
    :return: значение сплайна в точке
    """
    index = -100
    for i in range(n):
        if x[i] <= value <= x[i + 1]:
            index = i +1
            break
    splain_value = (((x[index] - value) ** 3 - (h ** 2) * (x[index] - value)) * m[index - 1]) / (6 * h) + (
            ((value - x[index - 1]) ** 3 - (h ** 2) * (value - x[index - 1])) * m[index]) / (6 * h) + (
                           (x[index] - value) * fun(x[index - 1])) / h + ((value - x[index - 1]) * fun(x[index])) / h
    return splain_value


In [19]:
def calculateSplain(a: float, b: float, n: int,
                    fun, diferSecond):
    """
    Расчет массивов методом прогонки
    :param a: левая граница
    :param b: правая граница
    :param n: количество разбиений
    :param fun: лямбда функция искомой функции
    :param diferSecond: лямбда второй производной
    :return: Матрицу коэффициентов, матрицу решений, количество разбиений, шаг
    """
    left_matrix, center_matrix, right_matrix, u_array, h, x_matrix, lambda_array = initParams(a, b, n,
                                                                                              diferSecond)
    for i in range(1, n - 1):
        left_matrix[i, i - 1] = h / 6  # Присваеваем коэф с
        left_matrix[i, i] = 2 * h / 3  # коэф а
        left_matrix[i, i + 1] = h / 6  # коэф b

        right_matrix[i] = (fun(x_matrix[i + 1]) - fun(x_matrix[i])) / h - (fun(x_matrix[i]) - fun(x_matrix[i - 1])) / h

    answer = np.linalg.solve(left_matrix, right_matrix)

    return x_matrix, answer, n, h


In [26]:
deltaMax =[0]
count = [5]
spines = [0]
deltaK = [0]*13
deltaEstimated = [0]*13

x_matrix, center_mas, n, h = calculateSplain(a=0.0, b=math.pi, n=6, fun=lambda x: math.sin(x),
                                             diferSecond=lambda x: -math.sin(x))
spine = splainThird(x_matrix, center_mas, h, 6,  x_matrix[len(x_matrix)//2] - h/2, fun=lambda x: math.sin(x))
spines[0] = spine
deltaMax[0] = abs(spine - math.sin( x_matrix[len(x_matrix)//2] - h/2))
count.append(count[-1]*2)

for i in range(12):
    x_matrix, center_mas, n, h = calculateSplain(a=0.0, b=math.pi, n=count[-1]+1, fun=lambda x: math.sin(x),
                                                 diferSecond=lambda x: -math.sin(x))
    spine = splainThird(x_matrix, center_mas, h, count[-1]+1, x_matrix[len(x_matrix)//2] - h/2, fun=lambda x: math.sin(x))
    spines.append(spine)
    deltaMax.append(abs(spine - math.sin( x_matrix[len(x_matrix)//2] - h/2)))
    count.append(count[-1]*2)

for i in range(11):
    deltaK[i] = (spines[i+1] - spines[i])/(spines[i + 2] - spines[i + 1])
count.pop()

for i in range(12):
    deltaEstimated[i] = (spines[i+1] - spines[i])/(2**4)

result = np.array([count, deltaMax, deltaEstimated, deltaK]).transpose()
df = pd.DataFrame(result, columns=['count','deltaMax', 'deltaEstimated', 'deltaK'])
df

,count,deltaMax,deltaPrice,deltaK
0,5.0,4.472573e-04,-7.431300e-04,-1.284986
1,10.0,2.567792e-05,5.783175e-04,4.000132
2,20.0,1.590317e-06,1.445746e-04,4.000008
3,40.0,9.916603e-08,3.614358e-05,4.000001
4,80.0,6.194297e-09,9.035893e-06,4.000000
5,160.0,3.870819e-10,2.258973e-06,4.000000
6,320.0,2.418654e-11,5.647433e-07,4.000000
7,640.0,1.506240e-12,1.411858e-07,4.000000
8,1280.0,1.341149e-13,3.529646e-08,4.000000
9,2560.0,4.551914e-14,8.824114e-09,4.000000
